In [1]:
%cd /content/drive/MyDrive/DeepLearning_HW2

/content/drive/MyDrive/DeepLearning_HW2


In [2]:
!ls

all_plants_data  archive.zip  hw#2  Plants_2


In [3]:
# %cd ./Plants_2/

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import os
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms
from torch.utils import data
from torch.autograd import Variable

In [17]:
IMG_WIDTH = 32  # width of image
IMG_HEIGHT = 32  # height of image
# BATCH_SIZE = 32  # batch size
# SEED = 0  # random seed
# DROPOUT = 0.5  # dropout probability
# LEARNING_RATE = 0.001    # learning rate
# NUM_EPOCHS = 10   # number of epochs
# CNN_MODEL_PATH = 'cnn_classification_model.pt'  # path for saved CNN model
# NUM_CLASSES = 2 

In [18]:
transformation = transforms.Compose([
        # resize
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        # transform to tensors
        transforms.ToTensor(),
        # normalize
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [19]:


train_folder = "./Plants_2/train/"
train_dataset = datasets.ImageFolder(root = train_folder, 
                transform = transformation)

train_loader = data.DataLoader(train_dataset, batch_size = 8, shuffle = True)

In [20]:
val_folder = "./Plants_2/valid/"
val_dataset = datasets.ImageFolder(root = val_folder, 
                transform = transformation)

val_loader = data.DataLoader(val_dataset, batch_size = 8, shuffle = True)

In [21]:
test_folder = "./Plants_2/test/"
test_dataset = datasets.ImageFolder(root = test_folder, 
                transform = transformation)

test_loader = data.DataLoader(test_dataset, batch_size = 8, shuffle = True)

In [22]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),           
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           
            nn.ReLU(),
			      nn.Conv2d(24, 48, 4, stride=2, padding=1),           
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Tanh(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [32]:
def create_model():
    autoencoder = Autoencoder()
    print_model(autoencoder.encoder, autoencoder.decoder)
    # if torch.cuda.is_available():
    #     autoencoder = autoencoder.cuda()
    #     print("Model moved to GPU in order to speed up training.")
    return autoencoder

def print_model(encoder, decoder):
    print("============== Encoder ==============")
    print(encoder)
    print("============== Decoder ==============")
    print(decoder)
    print("")

In [24]:
autoencoder = create_model()

============== Encoder ==============
Sequential(
  (0): Conv2d(3, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(12, 24, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(24, 48, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (5): ReLU()
)
============== Decoder ==============
Sequential(
  (0): ConvTranspose2d(48, 24, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): ConvTranspose2d(24, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): ConvTranspose2d(12, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (5): Tanh()
)

Model moved to GPU in order to speed up training.


In [25]:
def get_torch_vars(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [27]:
# criterion = nn.BCELoss()
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())

for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, _) in enumerate(train_loader, 0):
        inputs = get_torch_vars(inputs)
        # ============ Forward ============
        encoded, outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 5 == 4:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0

[1,     5] loss: 1.342
[1,    10] loss: 1.235
[1,    15] loss: 0.821
[1,    20] loss: 0.603
[1,    25] loss: 0.412
[1,    30] loss: 0.360
[1,    35] loss: 0.300
[1,    40] loss: 0.299
[1,    45] loss: 0.283
[1,    50] loss: 0.294
[1,    55] loss: 0.235
[1,    60] loss: 0.223
[1,    65] loss: 0.266
[1,    70] loss: 0.250
[1,    75] loss: 0.225
[1,    80] loss: 0.252
[1,    85] loss: 0.272
[1,    90] loss: 0.253
[1,    95] loss: 0.240
[1,   100] loss: 0.278
[1,   105] loss: 0.282
[1,   110] loss: 0.258
[1,   115] loss: 0.224
[1,   120] loss: 0.233
[1,   125] loss: 0.216
[1,   130] loss: 0.248
[1,   135] loss: 0.237
[1,   140] loss: 0.231
[1,   145] loss: 0.223
[1,   150] loss: 0.241
[1,   155] loss: 0.214
[1,   160] loss: 0.178
[1,   165] loss: 0.170
[1,   170] loss: 0.155
[1,   175] loss: 0.158
[1,   180] loss: 0.169
[1,   185] loss: 0.148
[1,   190] loss: 0.147
[1,   195] loss: 0.142
[1,   200] loss: 0.126
[1,   205] loss: 0.133
[1,   210] loss: 0.145
[1,   215] loss: 0.132
[1,   220] 

KeyboardInterrupt: ignored

In [28]:
print('Finished Training')
print('Saving Model...')
if not os.path.exists('./weights'):
    os.mkdir('./weights')
torch.save(autoencoder.state_dict(), "./weights/autoencoder.pkl")

Finished Training
Saving Model...


## use encoder of autoencoder for classification

In [29]:
class encoderAsClassifier(nn.Module):
    def __init__(self, autoencoder):
        super(encoderAsClassifier, self).__init__()
        self.encoder = autoencoder.encoder
        self.fc1 = nn.Linear(24 * 48 * 4, 256)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [33]:
autoencoder = create_model()

============== Encoder ==============
Sequential(
  (0): Conv2d(3, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(12, 24, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(24, 48, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (5): ReLU()
)
============== Decoder ==============
Sequential(
  (0): ConvTranspose2d(48, 24, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): ConvTranspose2d(24, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): ConvTranspose2d(12, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (5): Tanh()
)



In [34]:
autoencoder.load_state_dict(torch.load("./ae_model.pt"))

RuntimeError: ignored